# Statistiques d'utilisation des `publications`

- last_update: 2023-11-27

## objectif

Comprendre l'utilisation des publications par nos clients.

## TLDR

## setup

In [2]:
from pymongo.mongo_client import MongoClient
from utils import test_mongo_client, list_dbs, pull_db, RenderJSON
from utils import URI_CLUSTER_LOCAL, URI_CLUSTER_PROD
import pandas as pd
import plotly.express as px
import random
import seaborn as sns
import matplotlib.pyplot as plt
from enum import Enum

import plotly.express as px

## launch local instance mongodb
# launch-local-mongodb.sh

ImportError: attempted relative import with no known parent package

In [ ]:
## create mongo local client
client_local = MongoClient(URI_CLUSTER_LOCAL)
test_mongo_client(client_local)

#client_prod = MongoClient(URI_CLUSTER_PROD)
#test_mongo_client(client_prod)

client = client_local

In [ ]:
## refresh databases
#_ = pull_db(URI_CLUSTER_PROD, URI_CLUSTER_LOCAL, "org_afev")

In [ ]:
db_name = "org_amnesty"
#db_name = "org_albioma"
#pubs = get_pub(client, db_name)
#len(pubs)

In [ ]:
db_org = [d["name"] for d in list_dbs(client) if d["name"].startswith("org_")]

all_pubs = list()
for db_name in db_org:
    all_pubs += get_pubs(client, db_name)
pub_course = [p for p in all_pubs if is_pub_category(p, PUB_TYPE.COURSE)]
pub_adaptive = [p for p in all_pubs if is_pub_category(p, PUB_TYPE.ADAPTIVE)]
pub_single_module = [p for p in all_pubs if is_pub_category(p, PUB_TYPE.SINGLE_MODULE)]

## analyse

#### quel est le nombre total d'espace client ?

In [ ]:
nb_db_org = len(db_org)
print(f"Nombre d'espaces client: {nb_db_org}")

#### quel est le nombre total de publications ?

In [ ]:
all_pubs = list()
for db_name in db_org:
    all_pubs += get_pubs(client, db_name)

nb_pub = len(all_pubs)
print(f"total number of publications: {nb_pub}")

### Types de publication

In [ ]:
all_pub_type = [get_pub_type(p) for p in all_pubs]
all_pub_type = pd.Series(all_pub_type)
dist_pub_type = all_pub_type.value_counts().to_frame().reset_index(names="pub_type")

#### quel types de publications est-ce que Didask propose ?

In [ ]:
print("types de publications:")
for e in all_pub_type.unique():
    print(f"- {e}")

#### quelle est la distribution des publications par type ?

In [ ]:
dist_pub_type["pct"] = dist_pub_type["count"]/dist_pub_type["count"].sum()

In [ ]:
dist_pub_type

In [ ]:
ccc = dist_pub_type["pub_type"]
vvv = dist_pub_type["pct"]*100
ccc = [c.split(".")[1] for c in ccc]
plot_proportion_bar(cats=ccc, vals=vvv, title="Types de publication", is_pct=True, show_legend=False)

### mode de diffusion

- Les chiffres qui suivent sont restreints aux publications de type `Course`

#### quels sont les modes de diffusions possibles ?

In [ ]:
diff_modes = pd.Series([p["diffusionMode"] for p in pub_course if ("diffusionMode" in p)])
print("modes de diffusion:")
for dm in diff_modes.unique():
    print(f"- {dm}")

#### quelle est la distribution par mode de diffusion ?

In [ ]:
df_diff_modes = diff_modes.value_counts().to_frame()
df_diff_modes["pct"] = df_diff_modes["count"]/df_diff_modes["count"].sum()*100
df_diff_modes = df_diff_modes.reset_index(names='mode')
df_diff_modes
#df_diff_modes = df_diff_modes.melt(id_vars = "mode")


In [ ]:
cats = df_diff_modes["mode"]
vals = df_diff_modes["pct"]
title = "Mode de diffusion des publications"
is_pct = True
show_legend = False
plot_proportion_bar(cats=cats, vals=vals, title=title, is_pct=is_pct, show_legend=show_legend)